In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as partial
%matplotlib inline

In [2]:
# 导入训练数据和测试数据
df_train = pd.read_csv("train_data_public.csv")
df_test = pd.read_csv("test_public.csv")
df_train = df_train.drop(['id'],axis=1)
df_test = df_test.drop(['id'],axis=1)
df_train.head()

,text,BIO_anno,class
0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1


In [3]:
# 加载TokenEmbedding
from paddlenlp.embeddings import TokenEmbedding

# 初始化TokenEmbedding， 预训练embedding未下载时会自动下载并加载数据
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")

# 查看token_embedding详情
print(token_embedding)

c:\Users\Administer\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO 2022-08-25 19:06:37,301 download.py:117] unique_endpoints {''}
INFO 2022-08-25 19:06:37,304 download.py:250] Downloading w2v.baidu_encyclopedia.target.word-word.dim300.tar.gz from https://bj.bcebos.com/paddlenlp/models/embeddings/w2v.baidu_encyclopedia.target.word-word.dim300.tar.gz
100%|██████████| 694483/694483 [04:04<00:00, 2841.91it/s]
INFO 2022-08-25 19:10:42,094 download.py:287] Decompressing C:\Users\Administer\.paddlenlp\models\embeddings\w2v.baidu_encyclopedia.target.word-word.dim300.tar.gz...
[2022-08-25 19:11:19,087] [    INFO] - Loading token embedding...
[2022-08-25 19:11:51,152] [    INFO] - Finish loading embedding vector.
[2022-08-25 19:11:51,154] [    INFO] - Token Embedding info:       

Object   type: TokenEmbedding(635965, 300, padding_idx=635964, sparse=False)             
Unknown index: 635963             
Unknown token: [UNK]             
Padding index: 635964             
Padding token: [PAD]             
Parameter containing:
Tensor(shape=[635965, 300], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.24200200,  0.13931701,  0.07378800, ...,  0.14103900,
          0.05592300, -0.08004800],
        [-0.08671700,  0.07770800,  0.09515300, ...,  0.11196400,
          0.03082200, -0.12893000],
        [-0.11436500,  0.12201900,  0.02833000, ...,  0.11068700,
          0.03607300, -0.13763499],
        ...,
        [ 0.02628800, -0.00008300, -0.00393500, ...,  0.00654000,
          0.00024600, -0.00662600],
        [ 0.03962795,  0.02804082,  0.00407689, ..., -0.01970535,
          0.00478507,  0.00811509],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])


In [4]:
import paddle
import paddle.nn as nn
import paddlenlp

# 构建词袋模型
class BoWModel(nn.Layer):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder
        emb_dim = self.embedder.embedding_dim
        self.encoder = paddlenlp.seq2vec.BoWEncoder(emb_dim)
        self.cos_sim_func = nn.CosineSimilarity(axis=-1)

    def forward(self, text):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)

        # Shape: (batch_size, embedding_dim)
        summed = self.encoder(embedded_text)

        return summed

model = BoWModel(embedder=token_embedding)

In [33]:
from Tokenizer import Tokenizer
tokenizer = Tokenizer()
tokenizer.set_vocab(vocab=token_embedding.vocab)
train_dataset = []
for x in df_train['text']:
    train_dataset.append(tokenizer.text_to_ids(x))
train_dataset = np.c_[train_dataset,df_train['class']]

In [37]:
token_embedding.search("我")

array([[-7.07530e-02, -3.12400e-03,  1.84552e-01, -2.67849e-01,
        -8.77220e-02, -8.69740e-02,  1.36138e-01, -7.10070e-02,
        -3.80880e-02, -1.50260e-02, -1.57950e-02,  1.82037e-01,
        -3.69100e-02,  2.22004e-01,  2.50253e-01, -4.92480e-02,
        -1.22834e-01,  2.48200e-03,  2.35333e-01, -2.97400e-01,
         2.12635e-01, -9.12950e-02, -2.11589e-01, -3.88830e-02,
         3.64160e-02,  1.30371e-01, -7.92670e-02,  1.99963e-01,
         1.99222e-01,  1.38630e-02, -5.08490e-02,  2.74900e-02,
        -6.26530e-02,  1.35996e-01,  1.74689e-01, -4.88340e-02,
         5.69360e-02,  1.28470e-02,  2.49442e-01,  3.55970e-02,
         2.29490e-02,  4.67700e-03, -2.01530e-01, -7.96470e-02,
         2.49525e-01,  5.24710e-02, -1.78263e-01, -1.06662e-01,
         1.54286e-01,  8.03910e-02,  4.09948e-01,  1.48444e-01,
         9.40520e-02,  4.82200e-02,  9.85370e-02,  4.49790e-02,
         3.20980e-02, -4.57840e-02, -5.33584e-01,  2.00941e-01,
         3.52264e-01, -2.10759e-01, -1.4

ValueError: (InvalidArgument) The input tensor's dimension should be equal to the axis's size. But received input tensor's dimension is 1, axis's size is 3
  [Hint: Expected x_rank == axis_size, but received x_rank:1 != axis_size:3.] (at C:\home\workspace\Paddle_release\paddle/fluid/operators/transpose_op.cc:47)
  [operator < transpose2 > error]